In [25]:
import time 
import random
from random import randint
import numpy as np 
import pandas as pd 
from faker import Faker
from faker.providers import BaseProvider

In [2]:
def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

In [3]:
class CustomProvider:
    @classmethod
    def bdr_id(cls):
        faker = Faker()
        return faker.pystr_format("#########{{random_int}}") 

    @classmethod
    def iban_number(cls):
        faker = Faker()
        fake_iban = faker.country_code() + str(random_with_N_digits(2)) + faker.pystr(max_chars = 4).upper() + str(random_with_N_digits(6)) + str(random_with_N_digits(8))
        return fake_iban.upper() 
    
    @classmethod
    def id_class(cls):
        faker = Faker()
        return faker.pystr_format("#####{{random_int}}") 
    
    @classmethod
    def account_currency(cls):
        faker = Faker()
        return faker.pystr(max_chars = 3).upper()

    @classmethod
    def ten_letters(cls):
        faker = Faker()
        return faker.pystr(max_chars = 10).upper()
    
    @classmethod
    def two_letters(cls):
        faker = Faker()
        return faker.pystr(max_chars = 2).upper()
    
    @classmethod
    def one_letter(cls):
        faker = Faker()
        return faker.pystr(max_chars = 1).upper()
    
    @classmethod
    def num_compte(cls):
        faker = Faker()
        return faker.country_code() + str(random_with_N_digits(25))

In [4]:
faker = Faker()
faker.add_provider(CustomProvider())

In [5]:
faker.country_code()

'KN'

In [6]:
#getattr(faker.unique, faker.country_code())

In [8]:
test = []
for i in range(10):
    test.append(getattr(faker.unique, 'iban_number')())

In [ ]:
def anonymizer(data, list_anonymised_dict):
    """
    Args:
        data (pandas.DataFrame)
            dataframe to anonymise
        list_anonymised_dict (list of dictionary)
            list of dictionary containing key as columns to anonymise & value as Faker category
    Returns:
        data (pandas.DataFrame)
            dataframe anonymised
        mapping (dictionary)
            mapping containing key as original value & value as anonymised value
    """
    mapping = {}
    for anonymised_dict in list_anonymised_dict:
        category = anonymised_dict['category']
        for col in anonymised_dict['col']:
            faker_unique_list = []
            unique_value = list((data[col]).unique())
            faker = Faker()
            faker.add_provider(CustomProvider())
            for i in range(len(unique_value)): 
                faker_unique_list.append(getattr(faker.unique, category)())
            mapping_anonymised = dict(zip(unique_value, faker_unique_list))
            mapping[col] = mapping_anonymised
            data[col] = data[col].map(mapping_anonymised)
    return data, mapping

In [ ]:
def anonymizer_multi(data_dict, anonymised_list):
    """
    Args:
        data_dict (dictionary of pandas.DataFrame)
            dictionary of dataframe to anonymise
        anonymised_list (list of dictionary)
            list of dictionary containing key as columns to anonymise & value as Faker category
    Returns:
        data (pandas.DataFrame)
            dataframe anonymised
    """
    mapping_multi = {}
    for table_name in data_dict:
        mapping_multi[table_name] = {}
    for anonymised_dict in anonymised_list:
        faker = Faker()
        faker.add_provider(CustomProvider())        
        for col in anonymised_dict['col']:
            print('col', col)
            unique_value = []
            faker_unique_list = []
            for table_name in anonymised_dict['table_name_list']:
                print('table', table_name)
                unique_value += list((data_dict[table_name][col]).unique())

                for i in range(len(unique_value)): 
                    faker_unique_list.append(getattr(faker.unique, anonymised_dict['category'])())
            
            mapping_anonymised = dict(zip(unique_value, faker_unique_list))
            for table_name in anonymised_dict['table_name_list']:
                mapping_multi[table_name][col] = mapping_anonymised
                data_dict[table_name][col] = data_dict[table_name][col].map(mapping_anonymised)
    return data_dict, mapping_multi 

In [ ]:
def anonymizer_iban_prepro(data, list_col):
    """
    Args:
        data (pandas.DataFrame)
        list_col (list of string)
            list of col to anonymize
    Returns:
        data anonymized (pandas.DataFrame)
    """
    for col in list_col:
        print('col', col)
        data[f'{col}_2_letters'] = data[col].str[:2]
        
    return data 

In [ ]:
def anonymizer_iban_post(data, list_col):
    """
    Args:
        data (pandas.DataFrame)
        list_col (list of string)
            list of col to anonymize
    Returns:
        data anonymized (pandas.DataFrame)
    """
    for col in list_col:
        print('col', col)
        data[f'{col}_new'] = data[col]
        data = data.drop(columns = [col])
        data[col] = data[f'{col}_2_letters'] + data[f'{col}_new'].str[2:]
        print('Dropping col')
        data = data.drop(columns = [f'{col}_2_letters', f'{col}_new'])
    return data 

In [ ]:
data_dict = {'test': ['a', 'b', 'c']}

In [ ]:
df = pd.DataFrame(data_dict)

In [ ]:
df

In [ ]:
list_company_name_account_anonym = ['test']

In [ ]:
anonymised_dict_account = [{'col': list_company_name_account_anonym, 'category': 'company'}]

In [ ]:
anonymised_dict_account

In [ ]:
anonymised_dict_account[0]['col']

In [ ]:
for anonymised_dict in anonymised_dict_account:
    print(anonymised_dict)
    for col in anonymised_dict['col']:
        print('col:', col)

In [ ]:
(df, anonymised_dict_account)

In [ ]:
mapping = {}
for anonymised_dict in anonymised_dict_account:
    print('anonymised_dict:', anonymised_dict)
    category = anonymised_dict['category']
    for col in anonymised_dict['col']:
        print('col:', col)
        faker_unique_list = []
        unique_value = list((df[col]).unique())
        faker = Faker()
        faker.add_provider(CustomProvider())
        for i in range(len(unique_value)): 
            print('category', category)
            faker_unique_list.append(getattr(faker.unique, category)())
        mapping_anonymised = dict(zip(unique_value, faker_unique_list))
        mapping[col] = mapping_anonymised
        df[col] = df[col].map(mapping_anonymised)


In [ ]:
for i in range(len(unique_value)): 
    print('category')
    faker_unique_list.append(getattr(faker.unique, 'company')())

## FakeGenerator Class

In [68]:
class FakeGenerator():
    def __init__(self, num_rows : int, schema_list : str, custom_provder : CustomProvider):
        self.num_rows : int = num_rows
        self.schema_list : list = schema_list

    def generate_fake_data(self):
        data = pd.DataFrame()
        for schema in self.schema_list:
            faker = Faker()
            faker.add_provider(CustomProvider())
            faker_list = []
            for i in range(self.num_rows): 
                faker_list.append(getattr(faker, schema['category'])())
            data[schema['column']] = faker_list
        return data

In [98]:
class FakeGeneratorRefacto():
    def __init__(self, num_rows : int, schema_list : str, custom_provider : CustomProvider):
        self.num_rows : int = num_rows
        self.schema_list : list = schema_list
        self.custom_provider : CustomProvider = custom_provider

    def generate_fake_columns(self, schema):
        faker = Faker()
        faker.add_provider(self.custom_provider)
        return list(map(lambda x: getattr(faker, schema['category'])() , range(self.num_rows)))
        
    def generate_fake_data(self):
        data_columns = list(map(lambda schema: self.generate_fake_columns(schema), self.schema_list))
        data_columns = list(zip(*data_columns))
        fake_data = pd.DataFrame(data_columns, columns=[schema['column'] for schema in schema_list])
        return fake_data

In [ ]:
column_name = 'num_compte'
category_name = 'num_compte'
type = 'string'

In [35]:
num_rows = 5000
schema_list = [
{'column': 'name', 'category': 'name', 'type': type, 'unique': False},
{'column': 'address', 'category': 'address', 'type': type, 'unique': False},
{'column': 'job', 'category': 'job', 'type': type, 'unique': False},
{'column': 'company', 'category': 'company', 'type': type, 'unique': False},
]

In [56]:
def generate_fake_columns(schema):
    faker = Faker()
    faker.add_provider(CustomProvider())
    return list(map(lambda x: getattr(faker, schema['category'])() , range(10)))

In [50]:
data = pd.DataFrame()
map(lambda x : generate_columns(data, x), schema_list)

In [57]:
data_columns = list(map(lambda schema: generate_fake_columns(schema), schema_list))
data_columns = list(zip(*data_columns))

In [59]:
data = pd.DataFrame(data_columns, columns=[schema['column'] for schema in schema_list])

In [60]:
data

,name,address,job,company
0,Jennifer Phelps,"735 Watkins Road\nPort Kristina, DE 87881",Geophysical data processor,"Tate, Schaefer and Curtis"
1,April Miller,"293 Robert River\nStaceyside, AR 09487",Travel agency manager,"Fox, Foster and Brown"
2,Donald Perry,"927 Ralph Shoal\nJohnsonborough, RI 65240",Agricultural consultant,Freeman-Reeves
3,Stephen Riley,"4362 Villanueva Parkways\nEast Roberttown, ND ...",Clinical research associate,Snyder-White
4,Diane Ayala,"0238 Taylor Rapids Apt. 741\nEast Briannafurt,...",Facilities manager,"Hunter, Sharp and Meadows"
5,Lisa Roberts,835 Booker Estate Suite 287\nLake Elizabethbor...,Equities trader,Smith-Price
6,Tina Velasquez,"18324 Goodwin Terrace\nHernandezchester, CA 51779",Solicitor,Mercer-Powers
7,Sarah Peterson,"70897 Wood Loop\nSouth Jamesshire, PW 67522",Trading standards officer,"Barajas, Chandler and Davis"
8,Tina Gonzalez,"44971 Kenneth Mount\nCarlsonshire, PA 75377",Transport planner,Washington and Sons
9,Sarah Davis,"34832 Wade Passage Suite 699\nHillport, WV 35797",Hydrogeologist,Joseph and Sons


In [ ]:
data = pd.DataFrame()
for schema in schema_list:
    data[schema['column']] = faker_list

In [39]:
data

,name,address,job,company
0,Gwendolyn Hood,"97104 Kayla Locks\nNew Brenda, VA 33262",Local government officer,Shelton Group
1,Justin Lopez,"289 Brian Tunnel\nNorth Catherine, CT 86300",Acupuncturist,Wilkins and Sons
2,Sandra Pruitt,"02930 Douglas Mountain\nPort Barbarashire, VA ...",Television camera operator,Erickson-Valdez
3,Donald Davis,"2429 Garcia Cliff Apt. 639\nNorth Kathymouth, ...",Freight forwarder,Dickerson-Small
4,Terri Montes,"94137 Miranda Islands\nNew Steven, OH 25285",Glass blower/designer,Taylor-Barron
5,Dennis Henry,"190 Danielle Plaza\nPort Tinaburgh, CA 18640","Engineer, control and instrumentation",Rodgers-Brooks
6,Tina Huffman,"480 Misty Vista Apt. 716\nNorth Thomas, PW 15930",Television floor manager,Clark-Griffith
7,Marcus Jimenez,Unit 6511 Box 5259\nDPO AA 97363,Public relations account executive,Williams and Sons
8,Dale Mcintosh,"601 Rodney Junction\nSmithberg, VA 76858",Cabin crew,"Pace, Rodriguez and Lee"
9,Jeffrey Stephens,"87113 Jonathan Extension Apt. 644\nBrianshire,...",Data processing manager,Jordan PLC


In [89]:
num_rows = 5000

In [97]:
start = time.time()
original_data = FakeGenerator(num_rows, schema_list, CustomProvider()).generate_fake_data()
stop = time.time()
print(f'time generation : {stop - start}')

time generation : 1.2091639041900635


In [96]:
start = time.time()
test_data = FakeGeneratorRefacto(num_rows, schema_list, CustomProvider()).generate_fake_data()
stop = time.time()
print(f'time generation : {stop - start}')

time generation : 1.202030897140503


In [85]:
test_data.shape

(10, 4)

### List of Category Faker 
- Faire une liste des categories existantes dans Faker 

**address**
- address
- building_number
- city
- city_suffix
- country
- country_code
- current_country
- current_country_code()
- postcode
- street_address
- street_name
- street_suffix

**bank**
- aba
- bank_country
- bban
- iban
- swift
- swift8
- swift11

**company**
- bs
- catch_phrase
- company
- company_suffix

**currency:**
- cryptocurrency
- cryptocurrency_code
- cryptocurrency_name
- currency_code
- currency_name
- currency_symbol
- pricetag

**phone_number:**
- country_calling_code
- msisdn
- phone_number

**person:**
- first_name
- first_name_female
- first_name_male
- language_name
- last_name
- name
- suffix
- job

**geo:**
- coordinate
- latitude
- latlng
- longitude

**credit_card_number:**
- credit_card_expire
- credit_card_full
- credit_card_number
- credit_card_provider
- credit_card_security_code

### Unique Value 

In [62]:
getattr(faker.unique, 'name')()

'Miranda King'